# Create Pixel Spectrogram Dataset
This notebook downloads Sentinel data to produce inputs to a spectral-temporal pixelwise-classifier. 
Currently the notebook supports two time-steps, separated by a number of months assigned in MOSAIC_GAP, below. 

## Inputs
The notebook operates by loading a set of sampling sites from a geojson. If the geojson contains `Point` features, a bounding rect is constructed. If the geojson contains `Polygon` or `MultiPolygon` features, only pixels within the polygon will be extracted.

The `download_patch` script attempts to mask clouds. However, cloudy pixels and patches can still come through.

Pixels that fall outside of a polygon are also masked using a numpy masked array. These pixels are not stored in the output pixel arrays.

## Outputs

### Pixel Arrays:
The output list of arrays is saved as a pickle. The arrays are not normalized. The dimensionality of each array is  `[bands][num_time_steps]`, with num_time_steps hard-coded to 2. 
### Image Plot:
To log the data in a pixel array dataset, a grid of input images is exported along with the datset.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import descarteslabs as dl
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

from scripts import dl_utils
from scripts.viz_tools import band_descriptions, plot_image_grid

### Define Parameters for data extraction
### Attention: make sure to set appropriate label class!
Negative sites = 0, Positive sites = 1

In [ ]:
sampling_file = 'tpa_polygons'
data_dir = '../data/sampling_locations/'
label_class = 1

START_DATE = '2019-01-01'
END_DATE = '2021-01-01'
MOSAIC_PERIOD = 1  # the period over which to mosaic image data in months
MOSAIC_GAP = 6  # For spectrogram analysis, time gap between mosaics in number of mosaic periods

OUTPUT_DIR = f'../data/training_data/pixel_arrays_{MOSAIC_PERIOD}mo-mosaics_{MOSAIC_GAP}mo-gap'
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [ ]:
# Create or extract polygons from a sampling location
with open(os.path.join(data_dir, sampling_file + '.geojson'), 'r') as f:
    data = json.load(f)['features'] 
    
# Set rect width in pixels. Only required for point samples
num_pixels = 48
# Convert pixels to degrees. Heuristic, not geographically sound
# Better to go with slightly bigger patches that can then be cropped
rect_width = np.round((num_pixels / 100) / 111.32, 4)    

polygons = []
for feature in data:
    if feature['geometry']['type'] == 'Point':
        polygons.append(dl_utils.rect_from_point(feature['geometry']['coordinates'], rect_width))
    if feature['geometry']['type'] == 'MultiPolygon' or feature['geometry']['type'] == 'Polygon':
        polygons.append(feature['geometry'])
print(f'{len(polygons)} polygons loaded.')

### Download Sentinel data

In [ ]:
pairs = []
for polygon in tqdm(polygons):
    mosaics = dl_utils.download_mosaics(polygon, START_DATE, END_DATE, MOSAIC_PERIOD)
    pairs += dl_utils.pair(mosaics, MOSAIC_GAP)
print(len(pairs), "pairs of images extracted")

In [ ]:
def save_pixel_arrays(data, basepath, label_class):
    with open(basepath + '_pixel_arrays.pkl', "wb") as f:
        pickle.dump(data, f)
    with open(basepath + '_pixel_array_labels.pkl', "wb") as f:
        pickle.dump([label_class] * len(data), f)

In [ ]:
# View all images
figure_file_path = os.path.join(OUTPUT_DIR, f"{sampling_file} - Class {label_class}")
unpaired = [img for pair in pairs for img in pair]
plot_image_grid(unpaired, file_path=figure_file_path)

In [ ]:
# Create pixel arrays
pixel_arrays = []
for pair in tqdm(pairs):
    height, width, channels = pair[0].shape
    img_array = np.moveaxis(np.array(pair), 0, -1)
    img_array = img_array.reshape(height * width, channels, len(pair))
    pixel_arrays += [pixel for pixel in img_array if np.mean(pixel) > 0]

print(f"{len(pixel_arrays)} pixel arrays extracted")
basepath = os.path.join(OUTPUT_DIR, f"{sampling_file}_{START_DATE}_{END_DATE}")
save_pixel_arrays(pixel_arrays, basepath, label_class)

### Optional: plot the mean pixel spectra of the extracted dataset.
Process can take time with many samples

In [ ]:
all_pixels = np.moveaxis(np.array(pixel_arrays), -1, -2).reshape(2 * len(pixel_arrays), len(band_descriptions))
data = pd.DataFrame(all_pixels, columns=band_descriptions.keys()).melt(var_name='band', value_name='value')
plt.figure(figsize=(6,4), dpi=150, facecolor=(1,1,1))
sns.lineplot(x='band', y='value', data=data, ci="sd")
plt.title('Mean Value +/- SD')
plt.show()